In [ ]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

In [ ]:
print(torch.__version__ )

In [ ]:
#The datasets we will be using is FashionMNISH from torchvision.datasets
train_data = datasets.FashionMNIST(
    root = "data",#where to download data to
    download = True ,#whether we want to download the data
    train = True ,#whether we need training data or testing data
    transform = ToTensor(),#converting the image into the numbers(Tensor)
    target_transform=None#how do we want to transform the labels
)

In [ ]:
test_data = datasets.FashionMNIST(
    root = "data",
    download = True ,
    train = False ,
    target_transform = None ,
    transform=ToTensor(),
)

In [ ]:
len(train_data)

In [ ]:
class_names = train_data.classes
class_names

In [ ]:
image , label= train_data[0]
image , image.shape

In [ ]:
train_data.classes

In [ ]:
train_data.class_to_idx

In [ ]:
plt.imshow(image.squeeze() , cmap = "gray")
plt.title(class_names[label])

In [ ]:
torch.manual_seed(42)
fig = plt.figure(figsize = (9,9))
rows , cols = 4,4
for i in range(1 , rows*cols+1):
  random_index = torch.randint(0 ,len(train_data) , size = [1]).item()
  image , label = train_data[random_index]
  fig.add_subplot(rows , cols , i)
  plt.imshow(image.squeeze() , cmap = "gray")
  plt.title(class_names[label])

###PREPARE DATALOADERS

DataLoader turns our data into Python Iterable
Now our data is in the form of Python Iterable

Now we should convert our huge amount of data into data batches

Reasons for converting into Mini Batches
1.It gives our NN a time to update its gradient
2.Our computer wont be able to process this amount of data at a time
3.So we will split the 60000 training data set into each set containing 32 , A mini batch contains 32 datum


In [ ]:
#Prepare Data Loaders
from torch.utils.data import DataLoader
train_dataloader = DataLoader(shuffle = True , batch_size = 32 ,dataset = train_data)

test_dataloader = DataLoader(shuffle = False , batch_size = 32 , dataset = test_data)

In [ ]:
train_dataloader.batch_size

In [ ]:
print(f'The length of the Train Data Loader {len(train_dataloader)} and the size of batch is {train_dataloader.batch_size}')

print(f'The length of the Test Data Loader {len(test_dataloader)} and the size of batch is {test_dataloader.batch_size}')

In [ ]:
train_dataloader_image , train_dataloader_labels = next(iter(train_dataloader))
train_dataloader_image.shape , train_dataloader_labels.shape
print(len(train_dataloader))

In [ ]:
train_dataloaderimage , train_dataloaderlabel = next(iter(train_dataloader))
random_idx = torch.randint(0 , len(train_dataloader_image) , size = [1]).item()
img , label = train_dataloaderimage[random_idx] , train_dataloaderlabel[random_idx]
plt.imshow(img.squeeze() , cmap = "gray")
plt.title(class_names[label])
plt.axis = False

In [ ]:
x = train_dataloader_image[0]
x.shape

In [ ]:
#Now we arw going to build a base line model
#Now we have to use a Flatten Layer
flatten = nn.Flatten()
output = flatten(x)
print(f'The shape before flattening is {x.shape}')
#So the shape before flattening is [color_channels , height , width]
print(f'The shape after flattening is {output.shape}')
#After flattening the shape is [color_channels , (height*Width)]

In [ ]:
print(f'The Number of batches in the Training DataLoader : {train_dataloader.batch_size}')

In [ ]:
from torch import nn
class FashionMNISTmodel(nn.Module):
  def __init__(self,
               input_shape : int ,
               hidden_units : int ,
               output_shape : int):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(
            in_features = input_shape ,
            out_features = hidden_units ,
        ),
        nn.Linear(in_features= hidden_units ,
                  out_features = output_shape)
    )
  def forward(self , x) :
    return self.layer_stack(x)


In [ ]:
model = FashionMNISTmodel(
    input_shape = 28 * 28 ,
    hidden_units = 10 ,
    output_shape= len(class_names)
).to("cpu")
model

In [ ]:
loss_fun = nn.CrossEntropyLoss()
optim = torch.optim.SGD(
    params = model.parameters(),
    lr = 0.1
)

In [ ]:
def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

#Creating a function to time our experiment
Two of the main things what we will track is
1.Model Performance(Loss and Accuracy values)
2.How fast it runs

In [ ]:
from timeit import default_timer as timer

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def calculate_time(start : float ,
                   end : float ,
                   device : torch.device = device):
  total_time = end - start
  print('The total time taken is : {total_time} second')

In [ ]:
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [ ]:
from tqdm.auto import tqdm
torch.manual_seed(42)
epochs = 3
time_start = timer()
for epoch in tqdm(range(epochs)):
  train_loss = 0
  train_accuracy = 0
  for batch , (X , y) in enumerate(train_dataloader):
    y_pred = model(X)
    loss = loss_fun(y_pred , y)
    train_loss += loss.item()
    accuracy = accuracy_fn(y_true = y , y_pred = y_pred.argmax(dim=1))
    train_accuracy += accuracy
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  train_loss /= len(train_dataloader)
  train_accuracy /= len(train_dataloader)
  print(f'The Train Loss : {train_loss:.4f} and the Train Accuray : {train_accuracy:.4f}')
  test_accuracy = 0
  test_loss = 0
  model.eval()
  with torch.inference_mode() :
    for X , y in test_dataloader :
      test_preds = model(X)
      loss = loss_fun(test_preds , y)
      test_loss += loss.item()
      accuracy = accuracy_fn(y_true = y , y_pred = test_preds.argmax(dim = 1))
      test_accuracy += accuracy
  test_loss /= len(test_dataloader)
  test_accuracy /= len(test_dataloader)
  print(f'The Test Loss : {test_loss:.4f} and the Test Accuracy : {test_accuracy:.4f}')
end_time = timer()
print(calculate_time(start = time_start , end = end_time))

In [ ]:
model_0_time = calculate_time(start = time_start , end = end_time)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
from tqdm.auto import tqdm
torch.manual_seed(42)
train_time_on_cpu = timer()
epochs = 3
for epoch in tqdm(range(epochs)) :
  train_loss = 0
  for batch , (X,y) in enumerate(train_dataloader) :
    model.train()
    y_preds = model(X)
    loss = loss_fun(y_preds , y)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  train_loss /= len(train_dataloader)
  test_loss = 0
  test_acc = 0
  model.eval()
  with torch.inference_mode():
    for X , y in test_dataloader :
      test_preds = model(X)
      t_loss = loss_fun(test_preds , y)
      test_loss +=  t_loss.item()
      accuracy = accuracy_fn(y_true = y , y_pred = test_preds.argmax(dim = 1))
      test_acc += accuracy
    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)
  print(f'The TrainLoss : {train_loss:.4f} , TestLoss : {test_loss:.4f} , TestingAccuracy : {test_acc:.4f}')
end_time_on_cpu = timer()
print(f'The total time took to complete training and testing the model is : {end_time_on_cpu - train_time_on_cpu}')

In [ ]:
from timeit import default_timer as timer
from tqdm.auto import tqdm
def eval_model(model : torch.nn.Module ,
               loss_fun : torch.nn.Module ,
               data : torch.utils.data.DataLoader ,
               accuracy_fn) :
  test_loss = 0
  test_accuracy = 0
  model.eval()
  with torch.inference_mode():
    for X , y in tqdm(data) :
      test_preds = model(X)
      loss = loss_fun(test_preds , y)
      test_loss += loss.item()
      test_accuracy += accuracy_fn(y_true = y , y_pred = test_preds.argmax(dim = 1))
    test_loss /= len(data)
    test_accuracy /= len(data)
    return {"Model_name" : model.__class__.__name__ ,
            "Model_loss" : test_loss ,
            "Model_accuracy" : test_accuracy}
model0_results = eval_model(model = model , loss_fun = loss_fun , data = test_dataloader , accuracy_fn = accuracy_fn)

In [ ]:
model0_results

In [ ]:

print(len(train_data) + len(test_data))

In [ ]:
!nvidia-smi
import torch

In [ ]:
torch.cuda.is_available()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

In [ ]:
from torch import nn

###SECOND MODEL , WE ARE GOING TO BUILD AN ANOTHER MODEL , LINEAR AND NON LINEARITY . Our first model has only the Flatten Layer , and two of the Linear Layer but it performs a quite well and provides a good acuuracy . Now we will build a second model with a activation function RELU . This phase is called Experimentation

In [ ]:
class FashionMNISTNon(nn.Module) :
  def __init__(self ,
               input_shape : int ,
               hidden_units : int ,
               output_shape : int):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(),#which turns a tensor into a vector space
        nn.Linear(
            in_features = input_shape,
            out_features = output_shape
        ),
        nn.ReLU(),
        nn.Linear(
            in_features = hidden_units ,
            out_features = output_shape
        ),
        nn.ReLU()
    )
  def forward(self , x : torch.Tensor):
    return self.layer_stack(x)

In [ ]:
device

In [ ]:
class_names = train_data.classes
class_names

In [ ]:

class_index = train_data.class_to_idx
class_index

In [ ]:

#Creating the instance of the new model
torch.manual_seed(42)
model_1 = FashionMNISTNon(
    input_shape = 28*28 ,
    hidden_units = 10 ,
    output_shape = len(class_names)
).to(device)

In [ ]:
next(model_1.parameters()).device

In [ ]:
#Picking up a loss function , optimizer for our second model
loss_function = nn.CrossEntropyLoss() #measures how wrong out model is
optimizer = torch.optim.SGD(params = model_1.parameters() , lr = 0.1) #this imporoves the model's parameters which reduces the loss

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(shuffle = True , batch_size = 32 , dataset = train_data)
test_dataloader = DataLoader(shuffle = True , batch_size = 32 , dataset = test_data)

In [ ]:
print(f'The Batch size of the Train DataLoader : {train_dataloader.batch_size}')
print(f'The Batch size of the Test Dataloader : {test_dataloader.batch_size}')

In [ ]:
print(f'The number of batches in the Train Dataloader {len(train_data) / train_dataloader.batch_size}')
print(f'The number of batches in the Test Dataloader : {len(test_data) / test_dataloader.batch_size}')

In [ ]:
print(f'Therefore the total samples in the train data loader : {train_dataloader.batch_size * 1875}')
sum_1 = train_dataloader.batch_size * 1875
print(f'Therefore the total samples in the test data loader : {test_dataloader.batch_size * 312.5}')
sum_2 = test_dataloader.batch_size * 312.5
total_sum = sum_1 + sum_2
print(f'Therefore the total samples are : {total_sum}')

In [ ]:
#Picking up evaluation metrics
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
from timeit import default_timer as timer
from tqdm.auto import tqdm

In [ ]:

def train_step(
    model : torch.nn.Module,
    train_data : torch.utils.data.DataLoader,
    optimizer : torch.optim.Optimizer ,
    loss_fun : torch.nn.Module ,
    accuracy_fn ,
    device :torch.device = device
):
  model.train()
  model.to(device)
  epochs = 4
  for epoch in tqdm(range(epochs)) :
    train_loss = 0
    train_accuracy = 0
    for batch , (X , y) in enumerate(train_dataloader):
      X  = X.to(device)
      y = y.to(device)
      train_preds = model(X)
      loss = loss_fun(train_preds , y)
      train_loss += loss.item()
      accuracy = accuracy_fn(y_true = y , y_pred = train_preds.argmax(dim = 1))
      train_accuracy += accuracy
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    train_loss /= len(train_data)
    train_accuracy /= len(train_data)
    print(f'The Train Loss : {train_loss:.4f} and the Train Accuracy : {train_accuracy:.4f} for the Epoch {epoch+1}')

In [ ]:
train_step(
    model = model_1 ,
    train_data = train_dataloader ,
    optimizer = optimizer ,
    loss_fun = loss_function ,
    accuracy_fn= accuracy_fn ,
    device = device
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
from tqdm.auto import tqdm
def test_step(model : torch.nn.Module ,
              test_data : torch.utils.data.DataLoader ,
              loss_fun : torch.nn.Module ,
              accuracy_fn ,
              device : torch.device = device):
  model.to(device)
  model.eval()
  test_loss = 0
  test_accuracy = 0
  with torch.inference_mode() :
    for batch , (X , y) in enumerate(test_data) :
      X = X.to(device)
      y = y.to(device)
      test_preds = model(X)
      loss = loss_fun(test_preds, y)
      test_loss += loss.item()
      accuracy = accuracy_fn(y_true = y , y_pred = test_preds.argmax(dim=1))
      test_accuracy += accuracy
    test_loss /= len(test_data)
    test_accuracy /= len(test_data)
    print(f'The Test Loss : {test_loss:.4f} and the Test Accuracy : {test_accuracy:.4f}')

In [ ]:
test_step(model = model_1 , test_data = test_dataloader , loss_fun = loss_function , accuracy_fn = accuracy_fn)

In [ ]:
def eval_model1(model : torch.nn.Module ,
                data : torch.utils.data.DataLoader ,
                loss_fun : torch.nn.Module ,
                accuracy_fun ,
                device : torch.device = device ,
                ):
  test_loss = 0
  test_accuracy = 0
  model.to(device)
  model.eval()
  with torch.inference_mode() :
    for batch , (X , y) in enumerate(data) :
      X = X.to(device)
      y = y.to(device)
      test_preds = model(X)
      loss = loss_fun(test_preds , y)
      accuracy = accuracy_fun(y_true = y , y_pred = test_preds.argmax(dim = 1))
      test_loss += loss.item()
      test_accuracy += accuracy
    test_loss /= len(data)
    test_accuracy /= len(data)
    return {
        "Model_name" : model.__class__.__name__ ,
        "Model_loss": test_loss ,
        "Model_accuracy" : test_accuracy
    }

In [ ]:
model1_results = eval_model1(model = model_1 , data = test_dataloader , loss_fun = loss_function , accuracy_fun=accuracy_fn)

In [ ]:
model1_results

In [ ]:
###CONVOLUTIONAL NEURAL NETWORKS

In [ ]:
class FashionMNISTcnn(nn.Module):
  def __init__(self ,
               input_shape : int ,
               hidden_units : int ,
               output_units : int ,
                            ):
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(
            in_channels = input_shape ,
            out_channels = hidden_units ,
            kernel_size = 3 ,
            stride = 1 ,
            padding = 1
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels = hidden_units ,
            out_channels = hidden_units ,
            kernel_size = 3 ,
            stride = 1 ,
            padding = 1
        ),
        nn.MaxPool2d(kernel_size = 2),
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(
            in_channels = hidden_units ,
            out_channels = hidden_units ,
            kernel_size = 3 ,
            stride = 1 ,
            padding = 1
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels = hidden_units ,
            out_channels = hidden_units ,
            kernel_size = 3 ,
            stride = 1 ,
            padding = 1
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2),
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(
            in_features = hidden_units*7*7 ,
            out_features = output_units
        )
    )


  def forward(self , x):
    x = self.conv_block_1(x)
    #print(x.shape)
    x = self.conv_block_2(x)
    #print(x.shape)
    x = self.classifier(x)
    return x


In [ ]:
torch.manual_seed(42)
model_3 = FashionMNISTcnn(
    input_shape = 1 ,
    hidden_units = 10 ,
    output_units = len(class_names)
).to(device)

In [ ]:
#Now let us pass some dummy tensor through our model and then will observe the changes in the Shape
plt.imshow(image.squeeze() , cmap= "gray")
dummy_tensor_image = torch.randn(size = (1 ,28 ,28))
model_3(dummy_tensor_image.unsqueeze(0).to(device))

###STEPPING INTO CONV2D

In [ ]:
torch.manual_seed(42)
images = torch.randn(size = (32 , 3 , 64 , 64))
test_image = images[0]
conv_layer = nn.Conv2d(
    in_channels = 3 ,
    out_channels = 10,
    kernel_size = 3,
    stride = 1 ,
    padding = 1
)

###STEPPING THROUGH nn.MaxPool2d()

In [ ]:
#First let us pass our data into the Conv2d layer
conv_result = conv_layer(test_image)
print(f"The shape of the Image after passing through Conv2d layer {conv_result.shape}")

#Now lets create a MaxPool2d layer
max_pool_layer = nn.MaxPool2d(kernel_size = 2)
after_max_pool = max_pool_layer(conv_result)
print(f"The shape of the Image after passing through Conv2d Layer and MaxPool2d Layer is {after_max_pool.shape}")

In [ ]:
#Example of working of MaxPool2d layer
#Let us create a random tensor with shape (2,2) and will set the kernel_size as 2 in the max_pool layer
torch.manual_seed(42)
max_pool_layer = nn.MaxPool2d(kernel_size = 2)
random_tensor = torch.randn(1,1,2,2)
#Here (1,1,2,2) , here 1 is the batch size , 1 is the color channels , 2 is the height , 2 is the width
#Now let us pass this random_tensor to the maxpool2d layer with a kernel_size = 2

print(f"The size of the random tensor before going to MaxPool2d layer : {random_tensor.shape}")
print(random_tensor)
result_tensor_after_maxpool = max_pool_layer(random_tensor)
print(f"The shape of the Random Tensor after passing from MaxPool2d layer {result_tensor_after_maxpool.shape}")
print(f"The maximum value from the Random_Tensor is {result_tensor_after_maxpool}")

In [ ]:
#Now let us create a dataloader for our CNN , by using DataLoader

#This is for the Train Data
from torch.utils.data import DataLoader
cnn_train_data = DataLoader(shuffle = True , batch_size = 32 , dataset = train_data)
print(f"The Batch size of the Train Data : {cnn_train_data.batch_size}")
print(f"The total length of the Train Data : {len(train_data)}")
print(f"So the each batch of our cnn train data has {len(train_data)/ cnn_train_data.batch_size} images")

In [ ]:
#This is for the Test Data
cnn_test_data = DataLoader(shuffle = False , batch_size = 10 , dataset = test_data)
print(f"The Batch Size of the Test Data contains {cnn_test_data.batch_size} batches")
print(f"The total length of the Test Data : {len(test_data)}")
print(f"So each batch in the CNN test_data has {len(test_data)/cnn_test_data.batch_size} images")

###Now we will setup a Loss Function and a Optimiser for our CNN Model

In [ ]:
import torch.optim as optim
#Accuracy function is used for Evaluation Metrics
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc
loss_function_cnn = nn.CrossEntropyLoss()
optimizer_cnn = optim.SGD(params = model_3.parameters() , lr = 0.01)

In [ ]:
from tqdm.auto import tqdm
from timeit import default_timer as timer

In [ ]:
def train_step_cnn(
    model : torch.nn.Module ,
    train_data : torch.utils.data.DataLoader ,
    loss_fun : torch.nn.Module ,
    optimizer : torch.optim.Optimizer ,
    accuracy_fn ,
    device : torch.device = device
):
  model.train()
  model.to(device)
  epochs = 6
  for epoch in tqdm(range(epochs)):
    train_loss = 0
    train_accuracy = 0
    for batch , (X,y) in enumerate(train_data):
      X = X.to(device)
      y = y.to(device)
      train_preds = model(X)
      loss = loss_fun(train_preds , y)
      train_loss += loss.item()
      accuracy = accuracy_fn(y_true = y , y_pred = train_preds.argmax(dim=1))
      train_accuracy += accuracy
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    train_loss /= len(train_data)
    train_accuracy /= len(train_data)
    print(f"EPOCH : {epoch+1}")
    print(f"The Train Loss is {train_loss:.4f}")
    print(f"The Train Accuracy is {train_accuracy:.4f}")

In [ ]:
def test_step_cnn(model : torch.nn.Module ,
                  test_data : torch.utils.data.DataLoader ,
                  loss_fun : torch.nn.Module ,
                  optimizer : torch.optim.Optimizer ,
                  accuracy_fn ,
                  device : torch.device = device):
  model.eval()
  model.to(device)
  with torch.inference_mode() :
      test_loss = 0
      test_accuracy = 0
      for batch , (X,y) in enumerate(test_data):
        X = X.to(device)
        y = y.to(device)
        y_preds = model(X)
        loss = loss_fun(y_preds , y)
        test_loss += loss.item()
        accuracy = accuracy_fn(y_true = y , y_pred = y_preds.argmax(dim=1))
        test_accuracy += accuracy
      test_loss /= len(test_data)
      test_accuracy /= len(test_data)
      print(f"The Test Loss is {test_loss:.4f}")
      print(f"The Test Accuracy is {test_accuracy:.4f}")

In [ ]:
train_step_cnn(model = model_3 , train_data = cnn_train_data , loss_fun = loss_function_cnn , optimizer = optimizer_cnn , accuracy_fn=accuracy_fn)

In [ ]:

test_step_cnn(model = model_3 , test_data = cnn_test_data , loss_fun = loss_function_cnn , optimizer = optimizer_cnn , accuracy_fn=accuracy_fn)

In [ ]:

def eval_step_cnn(model : torch.nn.Module ,
                  test_data : torch.utils.data.DataLoader ,
                  loss_fun : torch.nn.Module ,
                  optimizer : torch.optim.Optimizer ,
                  accuracy_fn ,
                  device : torch.device = device):
  model.eval()
  model.to(device)
  with torch.inference_mode() :
      test_loss = 0
      test_accuracy = 0
      for batch , (X,y) in enumerate(test_data):
        X = X.to(device)
        y = y.to(device)
        y_preds = model(X)
        loss = loss_fun(y_preds , y)
        test_loss += loss.item()
        accuracy = accuracy_fn(y_true = y , y_pred = y_preds.argmax(dim=1))
        test_accuracy += accuracy
      test_loss /= len(test_data)
      test_accuracy /= len(test_data)
      return {
          "Model_name" : model.__class__.__name__ ,
          "Model_loss" : test_loss ,
          "Model_accuracy" : test_accuracy
      }

In [ ]:
model_cnn_results = eval_step_cnn(model = model_3 , test_data = cnn_test_data, loss_fun=loss_function_cnn ,optimizer=optimizer_cnn , accuracy_fn= accuracy_fn )

In [ ]:
model0_results

In [ ]:
model1_results

In [ ]:
model_cnn_results

###COMPARING OUR MODEL RESULTS

In [ ]:
import pandas as pd
compare_results = pd.DataFrame([
    model0_results,
    model1_results,
    model_cnn_results
])


In [ ]:
print(compare_results)

###MAKE AND EVALUATE RANDOM PREDICTIONS

In [ ]:
def make_predictions(
    model : torch.nn.Module ,
    data : list ,
    device : torch.device = device
):
  pred_probs = []
  model.to(device)
  model.eval()
  with torch.inference_mode():
    for sample in data :
      #Prepare a sample , add a batch dimension to the sample and send it to the model
      sample = torch.unsqueeze(sample , dim = 0).to(device)

      #Forward pass to the Model
      pred_logit = model(sample)

      #Get Prediction probability
      pred_prob = torch.softmax(pred_logit.squeeze() , dim = 0)

      #Store the pred_prob in the pred_probs list
      pred_probs.append(pred_prob.cpu())
  return torch.stack(pred_probs)

In [ ]:
test_data
from matplotlib import pyplot as plt

In [ ]:
import random
test_samples = []
test_labels = []

for sample , label in random.sample(list(test_data) , k = 9):
  test_samples.append(sample)
  test_labels.append(label)

pred_probs = make_predictions(model = model_3 , data = test_samples)
pred_classes = pred_probs.argmax(dim = 1)
plt.figure(figsize = (9,9))
ncols = 3
nrows = 3
for i , sample in enumerate(test_samples):
  plt.subplot(nrows , ncols ,i+1)
  plt.imshow(sample.squeeze()  ,cmap = "gray")
  pred_label = class_names[pred_classes[i]]
  truth_label = class_names[test_labels[i]]
  title_text = f"Pred:{pred_label}|Truth:{truth_label}"
  if pred_label == truth_label:
    plt.title(title_text , color = "green" , fontsize = 10)
  else :
    plt.title(title_text , color = "red" , fontsize = 10)


In [ ]:
!pip install nbformat